In [1]:
!pip install --upgrade pip
!pip install jupyter
!pip install scikit-learn
!pip install agentds-bench
!pip install xgboost
!pip install pandas
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# 1. Initialize Client and Load Data

import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, IsolationForest
from agentds import BenchmarkClient

# # 🔑 REPLACE WITH YOUR CREDENTIALS
# client = BenchmarkClient(
#     api_key="your-api-key-here",        # Get from your team dashboard
#     team_name="your-team-name-here"     # Your exact team name
# )

# Load data from PVC paths
print("📂 Loading Retail Banking Challenge 1 data...")

# Load transaction data
train_txns = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_train.csv")
test_txns = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_test.csv")

print(f"✅ Data loaded:")
print(f"   Train transactions: {train_txns.shape}")
print(f"   Test transactions: {test_txns.shape}")
print(f"   Train columns: {list(train_txns.columns)}")
print(f"   Test columns: {list(test_txns.columns)}")


📂 Loading Retail Banking Challenge 1 data...
✅ Data loaded:
   Train transactions: (152315, 11)
   Test transactions: (151587, 10)
   Train columns: ['Timestamp_dt', 'TxnID', 'Timestamp', 'SessionID', 'CustomerID', 'SrcAccount', 'DstAccount', 'Channel', 'MCC_Group', 'Amount', 'FraudLabel']
   Test columns: ['Timestamp_dt', 'TxnID', 'Timestamp', 'SessionID', 'CustomerID', 'SrcAccount', 'DstAccount', 'Channel', 'MCC_Group', 'Amount']


In [3]:
# 2. Tabular-Only Baseline Model and Predictions

# From data inspection - transaction columns:
# Timestamp_dt, TxnID, Timestamp, SessionID, CustomerID, SrcAccount, DstAccount, Channel, MCC_Group, Amount, FraudLabel (train only)

# Select features for baseline (exclude ID and target columns)
txn_features = ['Amount', 'Channel', 'MCC_Group']
print(f"📊 Using transaction features: {txn_features}")

# Prepare training data
X_train = train_txns[txn_features].fillna(0)
# Encode categorical features
for col in ['Channel', 'MCC_Group']:
    if col in X_train.columns:
        X_train[col] = pd.Categorical(X_train[col]).codes

X_train = X_train.astype(float)
y_train = train_txns['FraudLabel'].astype(int)  # Target variable

# Prepare test data
X_test = test_txns[txn_features].fillna(0)
# Encode categorical features (use same encoding as training)
for col in ['Channel', 'MCC_Group']:
    if col in X_test.columns:
        X_test[col] = pd.Categorical(X_test[col]).codes

X_test = X_test.astype(float)

# Train simple Random Forest classifier baseline
print("🤖 Training Random Forest classifier...")
model1 = RandomForestClassifier(n_estimators=100, random_state=42)
#model2 = XGBClassifier() XGBClassifier(n_estimators=100, max_depth=2, learning_rate=1, objective='binary:logistic')

model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Create submission file (format: TxnID,FraudLabel)
submission_df = pd.DataFrame({
    'TxnID': test_txns['TxnID'],
    'FraudLabel': predictions
})

# Save predictions
submission_df.to_csv("retailbanking_challenge1_predictions.csv", index=False)
print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
print(f"   Preview: {submission_df.head(3)}")
print(f"   Fraud rate: {predictions.mean():.3f} ({predictions.sum()} fraud cases out of {len(predictions)})")


📊 Using transaction features: ['Amount', 'Channel', 'MCC_Group']
🤖 Training Random Forest classifier...
✅ Predictions saved: 151587 predictions
   Preview:                                   TxnID  FraudLabel
0  df0c7ec6-eb0a-41bf-ade6-c4c9ebd941fe           0
1  273bf11f-aa15-4f61-9b2e-c47e7b960798           0
2  468da386-bda5-4380-93ba-362cf6d46bd5           0
   Fraud rate: 0.003 (461 fraud cases out of 151587)


In [4]:
# # 3. Submit Predictions

# # Submit predictions to the competition
# print("🚀 Submitting predictions...")

# try:
#     result = client.submit_prediction("Retailbanking", 1, "retailbanking_challenge1_predictions.csv")
    
#     if result['success']:
#         print("✅ Submission successful!")
#         print(f"   📊 Score: {result['score']:.4f}")
#         print(f"   📏 Metric: {result['metric_name']}")
#         print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
#     else:
#         print("❌ Submission failed!")
#         print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
# except Exception as e:
#     print(f"💥 Submission error: {e}")
#     print("🔧 Check your API key and team name are correct!")

# print("\n🎯 Next steps:")
# print("   1. Try incorporating relevant information outside this table!")
# print("   2. Move on to Retail Banking Challenge 2!")


In [7]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.6/620.6 MB 35.5 MB/s  0:00:09m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 26.3 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 75.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 64.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 80.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 77.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 64.3 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 66.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 13/26 [markdown]]]-data-server]  WARNING: The script markdown_py is installed in '/home/jovyan/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, i